Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
* Наименование вакансии.
* Предлагаемую зарплату (отдельно минимальную, максимальную и валюту).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
import requests
from pprint import pprint
import re
from bs4 import BeautifulSoup as bs
# https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=2&clusters=true&enable_snippets=true&text=Analyst+CRM&page=5
main_link = 'https://spb.hh.ru/search/vacancy'
vacancy_hh = 'Analyst CRM'
search_str = 6
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.193 Safari/537.36'
}


n_page = 0
vacancies = []
while n_page <= search_str - 1:
    params = {
        'L_is_autosearch':'false',
        'area': '2',
        'clusters':'true',
        'enable_snippets':'true',
        'text': vacancy_hh,
        'page': n_page

    }
    response = requests.get(main_link, params=params, headers=headers)


    if response.status_code == 200:
        dom = bs(response.text, 'html.parser')
        vacancy_list = dom.find_all('div', {'class': 'vacancy-serp-item__row vacancy-serp-item__row_header'})

        for vacancy in vacancy_list:
            vacancy_data = {}
            vacancy_name = vacancy.find('a').text
            vacancy_link = vacancy.find('a')['href']
            vacancy_salary = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})

            if not vacancy_salary:
                salary_min = None
                salary_max = None
                salary_currency = None
            else:
                vacancy_salary = vacancy_salary.getText().replace(u'\xa0', u'')
                vacancy_salary = re.split(r'\s|-', vacancy_salary)
                if vacancy_salary[0] == 'до':
                    salary_min = None
                    salary_max = float(vacancy_salary[1])
                    salary_currency = vacancy_salary[2]
                elif vacancy_salary[0] == 'от':
                    salary_max = None
                    salary_min = float(vacancy_salary[1])
                    salary_currency = vacancy_salary[2]
                else:
                    salary_max = float(vacancy_salary[1])
                    salary_min = float(vacancy_salary[0])
                    salary_currency = vacancy_salary[2]


            vacancy_data['name'] = vacancy_name
            vacancy_data['vacancy_link'] = vacancy_link
            vacancy_data['salary_min'] = salary_min
            vacancy_data['salary_max'] = salary_max
            vacancy_data['currency'] = salary_currency
            vacancies.append(vacancy_data)


    n_page += 1
pprint(vacancies)


[{'currency': None,
  'name': 'Аналитик CRM',
  'salary_max': None,
  'salary_min': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/39868701?query=Analyst%20CRM'},
 {'currency': 'руб.',
  'name': 'Руководитель отдела аналитики бизнес-процессов Битрикс24 и amoCRM',
  'salary_max': 140000.0,
  'salary_min': 60000.0,
  'vacancy_link': 'https://spb.hh.ru/vacancy/40119572?query=Analyst%20CRM'},
 {'currency': None,
  'name': 'Lead Business Analyst – CRM/BI',
  'salary_max': None,
  'salary_min': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/39676864?query=Analyst%20CRM'},
 {'currency': None,
  'name': 'Senior Business Analyst – CRM/BI',
  'salary_max': None,
  'salary_min': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/39676624?query=Analyst%20CRM'},
 {'currency': None,
  'name': 'Business Analyst – CRM/BI',
  'salary_max': None,
  'salary_min': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/39062328?query=Analyst%20CRM'},
 {'currency': 'руб.',
  'name': 'Analyst CRM',
  'sa

  'name': 'Email/CRM-маркетолог',
  'salary_max': None,
  'salary_min': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/37989452?query=Analyst%20CRM'},
 {'currency': None,
  'name': 'CRM-аналитик — Департамент развития бизнеса',
  'salary_max': None,
  'salary_min': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/39136922?query=Analyst%20CRM'},
 {'currency': None,
  'name': 'MySQL Database Analyst',
  'salary_max': None,
  'salary_min': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/39681237?query=Analyst%20CRM'},
 {'currency': 'руб.',
  'name': 'Специалист по маркетингу и рекламе',
  'salary_max': None,
  'salary_min': 60000.0,
  'vacancy_link': 'https://spb.hh.ru/vacancy/40264145?query=Analyst%20CRM'},
 {'currency': 'руб.',
  'name': 'Бизнес-аналитик в CRM-интегратор (Junior)',
  'salary_max': 45000.0,
  'salary_min': 20000.0,
  'vacancy_link': 'https://spb.hh.ru/vacancy/40197842?query=Analyst%20CRM'},
 {'currency': 'руб.',
  'name': 'Менеджер по продажам',
  'salary_max': 

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame(vacancies)

In [6]:
df[0:20]

,name,vacancy_link,salary_min,salary_max,currency
0,Аналитик CRM,https://spb.hh.ru/vacancy/39868701?query=Analy...,NaN,NaN,None
1,Руководитель отдела аналитики бизнес-процессов...,https://spb.hh.ru/vacancy/40119572?query=Analy...,60000.0,140000.0,руб.
2,Lead Business Analyst – CRM/BI,https://spb.hh.ru/vacancy/39676864?query=Analy...,NaN,NaN,None
3,Senior Business Analyst – CRM/BI,https://spb.hh.ru/vacancy/39676624?query=Analy...,NaN,NaN,None
4,Business Analyst – CRM/BI,https://spb.hh.ru/vacancy/39062328?query=Analy...,NaN,NaN,None
5,Analyst CRM,https://spb.hh.ru/vacancy/39364238?query=Analy...,100000.0,NaN,руб.
6,Заместитель руководителя отдела продаж,https://spb.hh.ru/vacancy/40275890?query=Analy...,NaN,NaN,None
7,Руководитель отдела продаж,https://spb.hh.ru/vacancy/40252475?query=Analy...,NaN,NaN,None
8,Бизнес-аналитик CRM/SAS,https://spb.hh.ru/vacancy/40348255?query=Analy...,NaN,NaN,None
9,Аналитик 1С (Стажер),https://spb.hh.ru/vacancy/40262092?query=Analy...,30000.0,30000.0,руб.
